In [22]:
import pandas as pd
import os
import statsmodels.formula.api as sm

from library import regulations
from library import characteristics
from library import analysis
from library import tables

In [23]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/descriptive/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data.csv'),
                  sep=",")


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Just keep eligible districts and charters from 2014-15

In [24]:
data = data[data.year == 'yr1415']
data = data[data.eligible == 1]

## District Characteristics

In [25]:
# Traditional public schools versus DOI
district = analysis.many_y_one_x(data = data, 
                                 y_list = characteristics.geography,
                                 y_labels = characteristics.labels,
                                 x = 'doi')
district
# note, I don't have the geographic characteristics of charter schools

,Characteristic,Control,Difference,Std. Error,P-value
0,Urban,0.06,-0.01,0.02,0.50
1,Suburban,0.25,-0.00,0.03,0.98
2,Town,0.24,0.05,0.03,0.16
3,Rural,0.45,-0.04,0.04,0.35


## Teacher Characteristics

In [26]:
teacher = analysis.many_y_one_x(data = data, 
                                 y_list = characteristics.teacher,
                                 y_labels = characteristics.labels,
                                 x = 'doi')
teacher

,Characteristic,Control,Difference,Std. Error,P-value
0,Ave. Experience in District,12.13,0.40,0.19,0.03
1,Ave. Experience Teaching,7.31,0.07,0.15,0.63
2,Teacher Turnover Ratio,20.00,-1.16,0.68,0.09
3,Student-Teacher Ratio,6.41,0.30,0.09,0.00


## Student Characteristics

In [27]:
student = analysis.many_y_one_x(data = data, 
                                 y_list = characteristics.student,
                                 y_labels = characteristics.labels,
                                 x = 'doi')
student

,Characteristic,Control,Difference,Std. Error,P-value
0,Percent Hispanic,0.51,-0.16,0.02,0.00
1,Percent White,0.41,0.13,0.02,0.00
2,Percent Black,0.05,0.02,0.01,0.04
3,Percent Econ. Disadvantaged,0.62,-0.08,0.01,0.00
4,Average STAAR Performance (Std.),-0.13,0.29,0.05,0.00


## To table

In [28]:
dfs = [district, teacher, student]
rows = [6, 15, 24]
tables.n_to_excel(file = table_path + 'DOI v. TPS.xlsx',
                 col = 2, row = 4, n = len(data[data.doi == 0]))
tables.n_to_excel(file = table_path + 'DOI v. TPS.xlsx',
                 col = 3, row = 4, n = len(data[data.doi == 1]))
for df, row in zip(dfs,rows):
    tables.var_diff_to_excel(file = table_path + 'DOI v. TPS v. Charter.xlsx', 
                             df = df, 
                             control_col = 'Control', 
                             diff_col = 'Difference', 
                             se_col = 'Std. Error', pvalue_col = 'P-value',
                             start_col = 2, start_row = row)

# Add charter columns

Separate into two datasets for ease of comparison - tps + doi and tps + charter

In [33]:
data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data.csv'),
                  sep=",")
data = data[data.year == 'yr1415']
data = data[(data.eligible == 1 ) | (data.charter == True)]
tpsdoi = data[data.eligible == 1]
tpscharter = data[(data.doi == 0) | (data.charter == True)]


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Some basic descriptives

In [34]:
num_dois = len(tpsdoi[tpsdoi.doi == 1])
num_tps = len(tpsdoi[tpsdoi.doi == 0])
print(str(len(tpsdoi)) + ' eligible districts, ' + str(num_dois), 'are DOIs and ' + str(num_tps) + ' have opted  out.')
print(str(len(tpscharter[tpscharter.charter == True])), "charter schools.")

973 eligible districts, 762 are DOIs and 211 have opted  out.
195 charter schools.


# Teacher characteristics

In [36]:
teacher = analysis.many_y_one_x(data = tpscharter, 
                                 y_list = characteristics.teacher,
                                 y_labels = characteristics.labels,
                                 x = 'charter')
teacher

,Characteristic,Control,Difference,Std. Error,P-value
0,Ave. Experience in District,12.13,-6.65,0.33,0.00
1,Ave. Experience Teaching,7.31,-4.94,0.25,0.00
2,Teacher Turnover Ratio,20.00,16.09,1.37,0.00
3,Student-Teacher Ratio,6.41,4.83,2.17,0.03


# Student Characteristics

In [37]:
student = analysis.many_y_one_x(data = tpscharter, 
                                 y_list = characteristics.student,
                                 y_labels = characteristics.labels,
                                 x = 'charter')
student

,Characteristic,Control,Difference,Std. Error,P-value
0,Percent Hispanic,0.51,-0.01,0.03,0.76
1,Percent White,0.41,-0.22,0.03,0.00
2,Percent Black,0.05,0.20,0.02,0.00
3,Percent Econ. Disadvantaged,0.62,0.06,0.02,0.01
4,Average STAAR Performance (Std.),-0.13,-0.31,0.09,0.00


# To Table

In [38]:
dfs = [teacher, student]
rows = [15, 24]
tables.n_to_excel(file = table_path + 'DOI v. TPS v. Charter.xlsx',
                 col = 4, row = 4, n = len(data[data.distischarter == 1]))
for df, row in zip(dfs,rows):
    tables.var_diff_to_excel(file = table_path + 'DOI v. TPS v. Charter.xlsx', 
                             df = df, 
                             control_col = 'Control', 
                             diff_col = 'Difference', 
                             se_col = 'Std. Error', pvalue_col = 'P-value',
                             start_col = 2, start_row = row,
                             added_third_col = True)